<a href="https://colab.research.google.com/github/lingchm/datascience-projects/blob/master/P5_Ecommerce_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-commerce Fraud Detection

**Summary of Fraud Detection Code Lab**
- Data is highly imbalanced
- Features of interval_after_signup and time related raw and aggregates are highly predictive of fraud
- Made actionable operation recommendations/proposal for business

**Background**

E-commerce websites often transact huge amounts of money. Whenever a huge amount of money is moved, there is a high risk of users performing fraudulent activities (e.g. using stolen credit cards, doing money laundry, etc.)

Company XYZ is an e-commerce site that sells electronic products. You have to build a model that predicts whether a user has a high probability of using the site to perform some illegal activity. You only have information about most user's first transaction on the site.

Goal: build a machine learning model that predicts the probability that the first transaction of a new user is fraudulent

**Data**

Table 1: "Fraud_Data" - information about each user's first transaction
* user_id: ID of the user. Unique by user
* signup_time: the time when the user created their account (GMT time)
* purchase_time: the time when the user bought the item (GMT time)
* device_id: the device id. Unique by the same physicall device
* source: user marketing channel (e.g. ads, SEO, Direct)
* browser: the browser used by the user
* sex: user's sec
* age: user's age
* ip_address: user's numeric IP address
* class: whether the activity was fraudulent (1) or not (0)

Table 2: "IpAddress_to_Country" - mapping each numeric IP address to its country
* lower_bound_ip_address: the lower bound of the numeric IP address for that country
* upper_bound_ip_address: the upper bound of the numeric IP address for that country
* country: if the IP address falls within the range, then the IP address belongs to the corresponding country

**Tasks:**
1. For each user, determine their country based on the numeric IP address
2. Build a model to predict whether an activity is fraudulent or not. Explain how different assumptions about the cost of false positives vs false negatives would impact the model.
3. How would you explain how the model is making the predictions from a user's perspectives? What kinds of users are more likely to be classified as at risk? What are their characteristics?
4. Let's say you now have this model which can be used to predict in real time if an activity is fraudulent or not. From a product perspective, how would you use it? What kind of different user experiences would you build based on the model output?

##Preprocessing

###Data Exploration

Reading in Data

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
!git clone https://github.com/loganlaioffer/fraudDetection.git

Cloning into 'fraudDetection'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [5]:
# Data also available on Google Drive folder
# !cd fraudDetection/
# !ls fraudDetection/
!pip install -U imbalanced-learn
!pip install pandas-profiling

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.6/dist-packages (0.7.0)


In [4]:
ipURL = 'https://github.com/loganlaioffer/fraudDetection/blob/master/IpAddress_to_Country.csv'
fraudURL = 'https://github.com/loganlaioffer/fraudDetection/blob/master/imbalancedFraudDF.csv'

ipToCountry = pd.read_csv('fraudDetection/IpAddress_to_Country.csv')
fraud_data = pd.read_csv('fraudDetection/imbalancedFraudDF.csv')

fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


Data Exploration

In [6]:
# Distribution of the label column
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

In [12]:
# Install pandas_profiling using the pip package manager by running:
import pandas_profiling

# Inline summary report without saving report as object
#pandas_profiling.ProfileReport(fraud_data)

In [ ]:
# simpler version without installing pandas_profiling
fraud_data.describe().transpose()

Missing values and Outliers

In [13]:
# count of NaN in each column
fraud_data.isna().sum()
# fraud_data.isnull().sum(axis = 0)

#HW: check count of fraud that is outliers

user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

In [ ]:
#Task 1: Identify country info based on ip_address
ipToCountry.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


###Map IP address to countries

In [14]:
# M1 Loop through transactions one by one
start = time.time()

# [] is list of T/F
# tmp is a df of shape n * 3 where n is 1 if found a match or 0 if no match
    
countries = []
for i in range(len(fraud_data)):
    ip_address = fraud_data.loc[i, 'ip_address']
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1: # found match
        countries.append(tmp['country'].values[0])
    else: # no match
        countries.append('NA')
        
fraud_data['country'] = countries
runtime = time.time() - start 

print("Lookup took", runtime, "seconds.")

Lookup took 177.27336978912354 seconds.


In [ ]:
ip_address = fraud_data.loc[6, 'ip_address']
tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
print(tmp)

       lower_bound_ip_address  upper_bound_ip_address        country
28203            1.686110e+09              1694498815  United States


In [ ]:
# M2 Binary Search
# iptocountry is already sorted and ranges are continuous
# TO DO 
# O(nlogn)

###Feature Engineering

Check if column user_id is unique for time related aggregates

In [ ]:
print(fraud_data.user_id.nunique()) #138376
print(len(fraud_data.index))        #138376
# all of the user_id has only the first 1 transaction, no way to do time based aggregates, 
# e.g. amount/counts in past 1 day for this user

138376
138376


Create some time-related features

In [ ]:
# time related features: can be done before split, as they has no interaction between other rows, solely based on other columns of the same row
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(
        fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_days_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear

# bed time operation
fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['purchase_days_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear
fraud_data['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour

fraud_data = fraud_data.drop(['user_id','signup_time','purchase_time'], axis=1)

In [ ]:
fraud_data.head()
# note there are NAs in country

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
0,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,4506682.0,55,82549,108,10031
1,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,17944.0,158,74390,159,5934
2,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,NA,492085.0,118,76405,124,50090
3,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,4361461.0,202,25792,252,67253
4,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,Canada,4240931.0,141,21783,190,29114


In [ ]:
print(fraud_data.source.value_counts())

SEO       55766
Ads       54913
Direct    27697
Name: source, dtype: int64


Q: What now? 
1. train/test split? 
2. convert categorical features e.g. device_id to numericals? 
 * If doing train/test split first (better), run the risk to have in test data categories that are not in training.
 * Solution 1 is to preserve a category called "others"
 * Solution 2 is to impute (KNN) to group with the most similar category
3. SMOTE sampling?

What if convert source column by OHE now?
it's a sneak peek of the levels of this column in test data, what if there is another new level "newsPaper" in source column in test, or one column in test data has larger range compared to that of train data? info leak, should not know this ahead of time in training


###Splitting Testing/Training Data

In [18]:
y = fraud_data['class']
X = fraud_data.drop(['class'], axis=1)

# split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

X_train.shape: (110700, 11)
y_train.shape: (110700,)


In [19]:
X_train['country'].value_counts(ascending=True)
#drawback: collision in the same bucket(no differentiation for these countries)

Fiji                  1
Benin                 1
Saint Martin          1
South Sudan           1
Gambia                1
                  ...  
United Kingdom     3253
Japan              5251
China              8876
NA                16275
United States     42348
Name: country, Length: 177, dtype: int64

In [ ]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
29343,12,OULPAZAFRFPXP,Ads,Chrome,M,42,3.690922e+09,Korea Republic of,3499664.0,183,67384,224,24648
12190,10,AIIWMFEYQQIEB,Ads,Opera,M,29,1.686759e+09,United States,6766039.0,5,78146,84,18585
19388,34,VUVETBUPCIWJE,Direct,Chrome,M,53,4.138429e+09,NA,5870515.0,197,81354,265,76669
89104,48,QCFULAJOYKFUU,Ads,Chrome,M,29,9.617337e+07,France,2145618.0,160,30920,185,16538
82082,44,IHRWLMIJMEEEU,Ads,FireFox,M,24,1.936025e+09,China,7079059.0,111,71897,193,66156


### Categorical Feature Encoding

Convert categorical features with high cadinality to numericals

In [20]:
# converting needs to be done after split
X_train = pd.get_dummies(X_train, columns=['source', 'browser'])# need to drop ['source', 'browser']? no, auto dropped by get_dummies 
X_train['sex'] = (X_train.sex == 'M').astype(int)

# Frequency encoding: 
# create new variables that count the number of times a device is shared

# the more a device is shared, the more suspicious
X_train_device_id_mapping = X_train.device_id.value_counts(dropna=False) # frequency count
X_train['n_dev_shared'] = X_train.device_id.map(X_train_device_id_mapping)# number of times device_id occurred in train data

# the more a ip is shared, the more suspicious
X_train_ip_address_mapping = X_train.ip_address.value_counts(dropna=False)
X_train['n_ip_shared'] = X_train.ip_address.map(X_train_ip_address_mapping)

# the less visit from a country, the more suspicious
X_train_country_mapping = X_train.country.value_counts(dropna=False)
X_train['n_country_shared'] = X_train.country.map(X_train_country_mapping)#lots of NAs in country column, #without dropna=False will produce nan in this col

X_train = X_train.drop(['device_id','ip_address','country'], axis=1)

In [21]:
### Technically, should use training data's mapping on testing data
# However, if the levels/values of the column in train and test data are pretty much the same(lots of overlap), e.g. country, 
# then we should apply the above 3 X_train_mappings (generated from X_train) on X_test(like below), 
# rather than using the new mapping generated from X_test

# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/target-encoding.html
# X_test['n_dev_shared'] = X_test.device_id.map(X_train_device_id_mapping) # if apply train mapping, most of the levels in test does not occur in train, so most are null after converting
# X_test['n_ip_shared'] = X_test.ip_address.map(X_train_ip_address_mapping)
# X_test['n_country_shared'] = X_test.country.map(X_train_country_mapping)

# but here device_id, ip_address has few overlap between train and test, if we apply the above 
# 3 X_train_mappings (generated from X_train) on X_test, then most of the X_test will be NaN
# this is the caveat of frequency encoding (not work well with many overlaps)


### Redo frequency encoding on testing data
# However, not recommended
X_test = pd.get_dummies(X_test, columns=['source', 'browser'])
X_test['sex'] = (X_test.sex == 'M').astype(int)

X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)

# Google: How to do categorical feature encoding with high cardinality?

In [ ]:
X_train.head()

,purchase_value,sex,age,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,n_dev_shared,n_ip_shared,n_country_shared
29343,12,1,42,3499664.0,183,67384,224,24648,1,0,0,1,0,0,0,0,1,1,3075
12190,10,1,29,6766039.0,5,78146,84,18585,1,0,0,0,0,0,1,0,1,1,42348
19388,34,1,53,5870515.0,197,81354,265,76669,0,1,0,1,0,0,0,0,1,1,16275
89104,48,1,29,2145618.0,160,30920,185,16538,1,0,0,1,0,0,0,0,1,1,2322
82082,44,1,24,7079059.0,111,71897,193,66156,1,0,0,0,1,0,0,0,1,1,8876


###Normalization


Variables needs to be brought to the same scale for models like LR with regularization (that are not tree based)


* normalize (min-max) to [0,1]
* standardize(StandardScaler) to normal, mu=0,var = 1 can < 0
* so we do normalize here because counts cannot be negative


In [22]:
# Compute the train minimum and maximum to be used for later scaling:
scaler = preprocessing.MinMaxScaler().fit(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']]) 
# print(scaler.data_max_)

# transform the training data and use them for the model training
X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

# apply the same scaler obtained from above on X_test, not fitting a brandnew scaler on test
X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])


In [23]:
X_train.n_dev_shared.value_counts(dropna=False)

0.0    105427
0.2      4774
0.4       324
0.6       124
0.8        45
1.0         6
Name: n_dev_shared, dtype: int64

In [24]:
X_test.n_dev_shared.value_counts(dropna=False)

0.0    27330
0.2      334
0.4       12
Name: n_dev_shared, dtype: int64

##Modeling

###Simple Logistic Regression Model

In [25]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

# predict on test
y_pred = logreg.predict(X_test)

ValueError: ignored

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)

# predicting everyone as 0 without adjusting weights

        pred_0  pred_1
true_0   27389       0
true_1     287       0


### Simple RF model

In [ ]:
classifier_RF = RandomForestClassifier(random_state=0)

classifier_RF.fit(X_train, y_train)

# predict class labels 0/1 for the test set
predicted = classifier_RF.predict(X_test)

# generate class probabilities
probs = classifier_RF.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0

# No false negatives, great!

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy_score is: : 0.9948692007515537
roc_auc_score is: : 0.7548596215347777
f1_score is: : 0.6712962962962962
confusion_matrix is: 
        pred_0  pred_1
true_0   27389       0
true_1     142     145
recall = 0.5052264808362369
precision = 1.0


### SMOTE sampling

Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality, and “blend” information from the test set into the training of the model. overfit! think about the case of simple oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and validation set, I could end up with the same observation in both datasets

https://imbalanced-learn.org/en/stable/install.html


In [ ]:
# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

pip install -U imbalanced-learn

# oversampling on only the training data, the right way! 
# sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling

smote = SMOTE(random_state=12)
x_train_sm, y_train_sm = smote.fit_sample(X_train, y_train)

unique, counts = np.unique(y_train_sm, return_counts=True)

print(np.asarray((unique, counts)).T)

[[     0 109572]
 [     1 109572]]


In [ ]:
#RF on smoted training data
classifier_RF_sm = RandomForestClassifier(random_state=0)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1]))
print('precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1]))

# worst precision? COnclude that resampling did not improve results...
# different AUC??

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy_score_sm is: : 0.9945440092498916
roc_auc_score_sm is: : 0.7533518568391924
f1_score_sm is: : 0.6544622425629291
confusion_matrix_sm is: 
        pred_0  pred_1
true_0   27382       7
true_1     144     143
recall or sens_sm = 0.49825783972125437
precision_sm = 0.9533333333333334


### Parameter tuning by GridSearchCV

Eval metrics for GridSearchCV over all fits upon combination of parameters and cv

In [ ]:
# Eval metrics to be calculated for each combination of parameters and cv, they are all calculated(by scoring), but not necessarily used to pick the optimal(by refit)
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score, pos_label=1)
}

In [ ]:
def grid_search_wrapper(model, parameters, refit_score='f1_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """

    grid_search = GridSearchCV(model, parameters, scoring=scorers, refit=refit_score,
                           cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # make the predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:, 1]
    
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
    print(cmDF)
    
    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print('recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1]))
    print('precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1]))

    return grid_search

Optimizing on f1_score on LR

In [ ]:
# C: inverse of regularization strength, smaller values specify stronger regularization
# penalty: LASSO's penalty term, l1 lasso l2 ridge
LRGrid = {"C" : np.logspace(-2,2,5), "penalty":["l1","l2"]} 
# param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logRegModel = LogisticRegression(random_state=0)

grid_search_LR_f1 = grid_search_wrapper(logRegModel, LRGrid, refit_score='f1_score')

# using optimized parameters, still not improved much

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Best params for f1_score
{'C': 0.1, 'penalty': 'l1'}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27386       3
true_1     278       9
	roc_auc_score is: : 0.7597098608854265
	f1_score is: : 0.06020066889632108
recall =  0.0313588850174216
precision =  0.75


Optimizing on f1_score on RF

In [ ]:
parameters = {        
  'max_depth': [None, 5, 15],
  'n_estimators' :  [10,150],
  'class_weight' : [{0: 1, 1: w} for w in [0.2, 1, 100]]
}

clf = RandomForestClassifier(random_state=0)

In [ ]:
grid_search_rf_f1 = grid_search_wrapper(clf, parameters, refit_score='f1_score')
# optimized parametrs: class_weight = 0.2
# exactly the same as the default
# RF works really well on imbalanced data

Best params for f1_score
{'class_weight': {0: 1, 1: 0.2}, 'max_depth': None, 'n_estimators': 150}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27389       0
true_1     142     145
	roc_auc_score is: : 0.7627425008361275
	f1_score is: : 0.6712962962962962
recall =  0.5052264808362369
precision =  1.0


In [ ]:
best_rf_model_f1 = grid_search_rf_f1.best_estimator_
best_rf_model_f1

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 0.2},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [ ]:
results_f1 = pd.DataFrame(grid_search_rf_f1.cv_results_)
results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortf1[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()

,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
9,1.0,0.527,0.69,1.0,0.527,0.690,5,"{0: 1, 1: 1}",150
7,1.0,0.527,0.69,1.0,1.000,1.000,None,"{0: 1, 1: 1}",150
13,1.0,0.527,0.69,1.0,1.000,1.000,None,"{0: 1, 1: 100}",150
3,1.0,0.527,0.69,1.0,0.527,0.690,5,"{0: 1, 1: 0.2}",150
11,1.0,0.527,0.69,1.0,0.584,0.737,15,"{0: 1, 1: 1}",150


Optimizing recall_score on RF

In [ ]:
grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')

Best params for recall_score
{'class_weight': {0: 1, 1: 100}, 'max_depth': 5, 'n_estimators': 150}

Confusion matrix of Random Forest optimized for recall_score on the test data:
        pred_0  pred_1
true_0   27146     243
true_1     132     155
	roc_auc_score is: : 0.7904661234456265
	f1_score is: : 0.4525547445255475
recall =  0.5400696864111498
precision =  0.38944723618090454


In [ ]:
best_RF_model_recall = grid_search_rf_recall.best_estimator_
best_RF_model_recall

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 100},
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=150, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [ ]:
# predict class labels for the test set
predictedBest_recall = best_RF_model_recall.predict(X_test)

# generate class probabilities
probsBest_recall = best_RF_model_recall.predict_proba(X_test)

results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)# recall score is different from above, as above is metric on test data, this is performance on cv data
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()
# recall is worse than default rf?? no this is on test, but train recall is better

,mean_test_precision_score,mean_test_recall_score,mean_test_f1_score,mean_train_precision_score,mean_train_recall_score,mean_train_f1_score,param_max_depth,param_class_weight,param_n_estimators
15,0.159,0.636,0.254,0.164,0.656,0.262,5,"{0: 1, 1: 100}",150
14,0.160,0.633,0.255,0.162,0.652,0.260,5,"{0: 1, 1: 100}",10
16,0.675,0.533,0.593,0.759,0.813,0.782,15,"{0: 1, 1: 100}",10
0,0.998,0.527,0.689,1.000,0.858,0.923,None,"{0: 1, 1: 0.2}",10
1,1.000,0.527,0.690,1.000,1.000,1.000,None,"{0: 1, 1: 0.2}",150


## Results Interpretation

### Variable Importance

In [ ]:
pd.DataFrame(best_rf_model_f1.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

,importance
interval_after_signup,0.394919
purchase_days_of_year,0.141148
purchase_seconds_of_day,0.081432
signup_seconds_of_day,0.078620
signup_days_of_year,0.059417
n_ip_shared,0.050463
purchase_value,0.044328
n_dev_shared,0.038656
age,0.038656
n_country_shared,0.027619


In [ ]:
# Based on the above var importance,
# the larger n_dev_shared, the higher rate of fraud
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF["n_dev_shared"],trainDF["class"])

class,0,1
n_dev_shared,,
0.0,104966,461
0.2,4403,371
0.4,152,172
0.6,37,87
0.8,13,32
1.0,1,5


In [ ]:
# the smaller action velocity (consecutive operations/actions of user), 
# interval_after_signup on frauds are significantly lower compared to legits
fraud_data.groupby("class")[['interval_after_signup']].mean()

,interval_after_signup
class,
0,5.191179e+06
1,2.570226e+06


In [ ]:
fraud_data.groupby("class")[['interval_after_signup']].median()#1
# more than half of fraud happened 1s after signed up -> robots?

,interval_after_signup
class,
0,5194911.0
1,1.0


In [ ]:
fraud_data[fraud_data['class'] == 1].head(100)

,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
136961,24,VLHGCDPFCICDA,SEO,Chrome,F,33,3.432126e+09,1,United States,3327952.0,218,80113,257,38465
136962,14,YLUQSRNYYIPXU,Ads,Chrome,M,40,3.905319e+09,1,NA,1.0,12,4207,12,4208
136963,63,ABUBCQDATQMQH,Ads,FireFox,F,46,5.505670e+08,1,United States,7640070.0,49,40723,137,77593
136964,34,QHEODGCAVJKIQ,SEO,Chrome,M,37,9.408096e+08,1,United States,1.0,12,77710,12,77711
136965,76,DAKVYHKIEYRBH,SEO,Chrome,F,48,6.361041e+08,1,Hungary,1.0,10,48421,10,48422
136966,32,ESANFBTIVMNHX,Ads,IE,M,30,3.875475e+09,1,NA,5721252.0,176,53824,242,72676
136967,95,HIAMXITLJWYCT,SEO,FireFox,M,42,3.786924e+09,1,NA,10160108.0,9,33511,126,84819
136968,13,BQTPLJBGYXQYX,Ads,IE,M,32,2.463262e+09,1,Austria,1.0,12,29576,12,29577
136969,15,BWSMVSLCJXMCM,Direct,IE,F,39,2.937899e+09,1,Japan,1.0,7,61065,7,61066
136970,26,HPPSDIRGUSSTB,Direct,Opera,M,31,6.471261e+08,1,United States,1.0,1,80617,1,80618


Insights
1. the larger n_dev_shared, the higher rate of fraud
2. the smaller action velocity, higher probabiity of fraud
3. More than half of fraud happend 1s after sign up

###How to use predictions

Create a score (1-10) to indicate likelihood of being fraud:
* green: 1 - 3 pass
* grey: 4-7 need manual investigation
* red: 8,9 decline

In [ ]:
t = (10 * probsBest_recall[:, 1]).astype(int)
unique, counts = np.unique(t, return_counts=True)

print(np.asarray((unique, counts)).T)

[[    1     1]
 [    2 24555]
 [    3  2623]
 [    4    99]
 [    5   177]
 [    6    76]
 [    7     1]
 [    8    20]
 [    9   124]]


In [ ]:
#HW: try XGBOOST and compare the performance of RF

In [ ]:
#HW: binary search

In [ ]:
#HW: Logistic regression parameter tuning using class_weights